## Create Edits that match post edit class distribution of target class to real edits

In [121]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from datetime import datetime

In [110]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, list_to_dict, ensure_dir, informal_log
from utils.df_utils import load_and_preprocess_csv
from utils.visualizations import histogram
from utils.model_utils import prepare_device, quick_predict
from parse_config import ConfigParser

import datasets.datasets as module_data
import model.model as module_arch
import model.metric as module_metric
import model.loss as module_loss

In [37]:
# Define constants, paths
class_list_path = os.path.join('metadata', 'cinic-10', 'class_names.txt')

config_path = 'configs/copies/edit_experiments/cinic10_imagenet-bump_edit.json'

In [44]:
# Load config file, models, and dataloader
class_list = read_lists(class_list_path)
class_idx_dict = list_to_dict(class_list)

config_dict = read_json(config_path)

device, device_ids = prepare_device(config_dict['n_gpu'])

### Get the Target Class Distribution Across All Edits for Specific Class

In [55]:
class_name = 'airplane'
target_class_idx = class_idx_dict[class_name]
n_select = 100
timestamp = '0127_103716'

root_dir = os.path.join('saved', 'edit', 'trials', 'CINIC10_ImageNet-VGG_16', '{}' + '_{}'.format(n_select), timestamp)
csv_path_template = os.path.join(root_dir, 'results_table.csv')


In [28]:
def save_class_distribution(csv_path, 
                            target_class_idx,
                            target_class_name,
                            show=False):
    
    df = load_and_preprocess_csv(
        csv_path=csv_path,
        drop_duplicates=['ID']
    )
    class_distribution = df['Post Class Dist'].to_numpy()
    class_distribution = np.stack(class_distribution, axis=0)
    target_class_distribution = class_distribution[:, target_class_idx]

    target_class_bins = np.bincount(target_class_distribution)

    histogram_save_path = os.path.join(os.path.dirname(csv_path), 'graphs', 'summary', 'target_class_distribution.png')
    title = 'Post Edit {} Class Distribution for {} Edits'.format(target_class_name, target_class_name)
    xlabel = 'Num. {} Predictions Post Edit'.format(target_class_name)
    ylabel = 'Num. Edits'
    
    bin_values, bins, _ = histogram(
        data=target_class_distribution,
        n_bins=50,
        save_path=histogram_save_path,
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        show=show)
    
    bin_tuples = []
    for bin_idx in range(len(bins) - 1):
        bin_tuples.append((bins[bin_idx], bins[bin_idx+1]))

    save_data = {
        "n_target_predictions": target_class_distribution,
        "histogram_bin_values": bin_values,
        "histogram_bins": bins
    }
    target_class_dist_save_path = os.path.join(os.path.dirname(csv_path), 'target_class_distribution.pth')
    torch.save(save_data, target_class_dist_save_path)
    
    print("Saved target class distribution & histogram data to {}".format(target_class_dist_save_path))
    
    plt.close('all')

#### Loop over all classes and save histograms and distributions

In [29]:
for class_name in class_list:
    csv_path = csv_path_template.format(class_name)
    save_class_distribution(
        csv_path=csv_path,
        target_class_idx=class_idx_dict[class_name],
        target_class_name=class_name,
        show=False)
    

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


50
50
Saved target class distribution & histogram data to saved/edit/trials/CINIC10_ImageNet-VGG_16/airplane_100/0127_103716/target_class_distribution.pth


In [30]:
dic = torch.load('saved/edit/trials/CINIC10_ImageNet-VGG_16/airplane_100/0127_103716/target_class_distribution.pth')
print(dic['histogram_bins'])

[ 8732.    9620.98 10509.96 11398.94 12287.92 13176.9  14065.88 14954.86
 15843.84 16732.82 17621.8  18510.78 19399.76 20288.74 21177.72 22066.7
 22955.68 23844.66 24733.64 25622.62 26511.6  27400.58 28289.56 29178.54
 30067.52 30956.5  31845.48 32734.46 33623.44 34512.42 35401.4  36290.38
 37179.36 38068.34 38957.32 39846.3  40735.28 41624.26 42513.24 43402.22
 44291.2  45180.18 46069.16 46958.14 47847.12 48736.1  49625.08 50514.06
 51403.04 52292.02 53181.  ]


### For a specific class, make bump edits that match the distribution of target class prediction

In [93]:
# Load datasets
config = ConfigParser(config_dict)
data_loader_args = dict(config.config["data_loader"]["args"])
dataset_args = dict(config["dataset_args"])

val_image_paths = read_lists(config_dict['dataset_paths']['valid_images'])
val_labels = read_lists(config_dict['dataset_paths']['valid_labels'])
val_paths_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=val_image_paths,
        labels=val_labels,
        return_paths=True,
        **dataset_args
    ),
    **data_loader_args
)

# Obtain loss function and metric functions
loss_fn = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

In [94]:
# Load model
layernum = config.config['layernum']
model = config.init_obj('arch', module_arch, layernum=layernum, device=device)
model = model.eval()


In [67]:
def predict_with_bump(data_loader,
                      model,
                      target_class_idx,
                      bump_amount,
                      loss_fn,
                      metric_fns,
                      device,
                      save_path=None):
    '''
    Run the model on the data_loader, calculate metrics, and log

    Arg(s):
        data_loader : torch Dataloader
            data to test on
        model : torch.nn.Module
            model to run
        loss_fn : module
            loss function
        metric_fns : list[model.metric modules]
            list of metric functions
        device : torch.device
        save_path : str or None
            if not None, save metrics to save_path

    Returns :
        log : dict{} of metrics
    '''

    # Hold data for calculating metrics
    outputs = []
    targets = []

    # Ensure model is in eval mode
    if model.training:
        model.eval()

    with torch.no_grad():
        for idx, item in enumerate(tqdm(data_loader)):
            if len(item) == 3:
                data, target, path = item
            else:
                data, target = item
            data, target = data.to(device), target.to(device)
            output = model(data)

            # Store outputs and targets
            outputs.append(output)
            targets.append(target)

    # Concatenate predictions and targets
    outputs = torch.cat(outputs, dim=0)
    targets = torch.cat(targets, dim=0)
    
    # Adjust output softmax by bump amount
    outputs[:, target_class_idx] += bump_amount

    # Calculate loss
    loss = loss_fn(outputs, targets).item()
    n_samples = len(data_loader.sampler)
    log = {'loss': loss}

    # Calculate predictions based on argmax
    predictions = torch.argmax(outputs, dim=1)

    # Move predictions and target to cpu and convert to numpy to calculate metrics
    predictions = predictions.cpu().numpy()
    targets = targets.cpu().numpy()

    # Calculate metrics
    log = module_metric.compute_metrics(
        metric_fns=metric_fns,
        prediction=predictions,
        target=targets)

    if save_path is not None:
        ensure_dir(os.path.dirname(save_path))
        torch.save(log, save_path)

    return log

In [136]:
def match_bump_edits(data_loader,
                     model,
                     loss_fn,
                     metric_fns,
                     device,
                     bin_lows,
                     target_class_idx,
                     save_dir,
                     debug=True):
    '''
    Get bins
    for each bin, get the mean
    until the target class predictions on val match the mean,
        adjust the target
    store (mean, bump amount)
    '''
    # save_dir = os.path.dirname(str(config.save_dir))
    informal_log_path = os.path.join(save_dir, 'informal_log.txt')
    bump_dictionary = {}
    
    n_bins = len(bin_lows) - 1
    
    n_bins_left = n_bins
    min_bin = bin_lows[0]
    bin_width = bin_lows[1] - bin_lows[0]
    
    bin_means = []
    bump_amount = 0.2
    delta = 0.1
    delta1 = 1.25
    
    bumps = [-1 for i in range(n_bins)]
    class_predictions = [-1 for i in range(n_bins)]
    logs = [None for i in range(n_bins)]
    
    cur_bump_idx = 0
    n_itr_on_current = 1
    max_itr_on_current = 20
    cur_itr = 0
    max_delta = 5
    delta_reset = 0.2
    delta1_reset = 1.25
    while cur_bump_idx < n_bins:
        cur_itr += 1
        # Reset delta if it drops below 0 or above max_delta
        if delta <= 0 or delta > max_delta:
            delta = delta_reset
        if delta1 <= 1 or delta1 > 2:
            delta1 = delta1_reset
        if debug:
            informal_log("[{}]***NEW ITR {}***".format(
                datetime.now().strftime(r'%m%d_%H%M%S'), cur_itr), informal_log_path)
        if n_itr_on_current > max_itr_on_current:
            if debug:
                informal_log("Max iter of {} on current reached.".format(max_itr_on_current), informal_log_path)
                informal_log("Cur_bump_idx: {} goal n pred: {} actual n_pred: {}"
                      .format(cur_bump_idx, bin_lows[cur_bump_idx], target_class_predictions), informal_log_path)
                informal_log("Cur delta: {} ".format(delta), informal_log_path)
            break
        if bumps[cur_bump_idx] > -1:
            bump_amount = bumps[cur_bump_idx]
            cur_bump_idx += 1
            
            continue
        
        log = predict_with_bump(
            data_loader=data_loader,
            model=model,
            loss_fn=loss_fn,
            metric_fns=metric_fns,
            device=device,
            target_class_idx=target_class_idx,
            bump_amount=bump_amount,
            save_path=None)
        
        # Obtain num. predictions for target class and determine bin idx
        post_class_distribution = log['predicted_class_distribution']
        target_class_predictions = post_class_distribution[target_class_idx]
        bin_idx = math.floor((target_class_predictions - min_bin) / bin_width)
        
        # Check for out of bounds/already visited bin index
        # if bin_idx < 0:
        #     delta = delta * (-1 * bin_idx) * n_itr_on_current
        #     # bump_amount += delta
        #     bump_amount *= delta1
        #     n_itr_on_current += 1
        #     informal_log("bin_idx < cur_bump_idx ({} and {}) delta={} new bump_amount: {}".format(
        #         bin_idx, cur_bump_idx, delta, bump_amount), informal_log_path)
        #     continue
        # elif bin_idx < cur_bump_idx:
        #     delta *= 2
        #     bump_amount = bumps[cur_bump_idx-1] + delta #* n_itr_on_current
        #     n_itr_on_current += 1
        #     informal_log("bin_idx < cur_bump_idx ({} and {}) delta={} new bump_amount: {}"
        #           .format(bin_idx, cur_bump_idx, delta, bump_amount), informal_log_path)
        #     continue
        while bin_idx < cur_bump_idx:
            # if cur_bump_idx == 0:
            #     bump_amount *= delta1
            # else:
            #     bump_amount = bumps[cur_bump_idx - 1] * delta1
            #     delta1 *= 2
            bump_amount *= delta1
                
            informal_log("bin_idx < cur_bump_idx ({} and {}) delta1={} new bump_amount: {}".format(
                bin_idx, cur_bump_idx, delta1, bump_amount), informal_log_path)
                
            log = predict_with_bump(
                data_loader=data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                target_class_idx=target_class_idx,
                bump_amount=bump_amount,
                save_path=None)

            # Obtain num. predictions for target class and determine bin idx
            post_class_distribution = log['predicted_class_distribution']
            target_class_predictions = post_class_distribution[target_class_idx]
            bin_idx = math.floor((target_class_predictions - min_bin) / bin_width)
                
            
            # continue
        if bin_idx >= n_bins:
            # delta = delta * (bin_idx - n_bins + 1)
            # delta *= 2
            if cur_bump_idx == 0:
                bump_amount /= delta1
                # bump_amount = max(bump_amount - delta, bumps[cur_bump_idx-1] + delta)
            else:
                bump_amount = (bump_amount + bumps[cur_bump_idx - 1]) / 2
                # bump_amount -= delta
            informal_log("bin_idx > n_bins ({}) delta1={} new bump_amount: {}".format(
                bin_idx, delta1, bump_amount), informal_log_path)
            continue
            
        # Sanity check correct bin_idx
        assert target_class_predictions >= bin_lows[bin_idx] and \
               target_class_predictions <= bin_lows[bin_idx+1]
        
        # If bump value for this bin yet, assign it
        if bumps[bin_idx] == -1:
            bumps[bin_idx] = bump_amount
            class_predictions[bin_idx] = target_class_predictions
            logs[bin_idx] = log
            informal_log("assigned bin_idx {}".format(bin_idx), informal_log_path)
            informal_log("Cur bump_idx: {} bump amount: {} target_class_pred: {} bin_idx: {}".format(
                cur_bump_idx, bump_amount, target_class_predictions, bin_idx), informal_log_path)
        
        # Check if this is our current bin we're trying to fill
        if bin_idx == cur_bump_idx:
            cur_bump_idx += 1
            # bump_amount += delta
            bump_amount *= delta1
            n_itr_on_current = 1
            # delta = 0.1
            if debug:
                informal_log("updating cur_bump_idx to {}, bump_amount to {}"
                             .format(cur_bump_idx, bump_amount), informal_log_path)
            
        # If we're not at the current cell,
        else:
            # Decrease bump amount by half
            if cur_bump_idx == 0:
                bump_amount /= 2
            # Set bump amount to something in the middle between current and last successful
            else:
                bump_amount = (bump_amount + bumps[cur_bump_idx-1]) / 2 
            if debug:
                informal_log("Overshot to bin {}. Decreasing bump amount to {} aiming for bin {}".format(
                    bin_idx, bump_amount, cur_bump_idx), informal_log_path)
        informal_log("bumps: {}".format(bumps), informal_log_path)

        
        # bump_amount += delta
        # n_bins_left -= 1
        
        # return log
        # if not os.path.isfile(save_path):
        #     torch.save(log, save_path)
    save_data = {
        'bump_amounts': bumps,
        'target_class_predictions': class_predictions,
        'metrics': logs
    }
    save_path = os.path.join(save_dir, 'bumps_preds_metrics.pth'.format())
    torch.save(save_data, save_path)
    print("Saved data to {}".format(save_path))
            

In [137]:
target_class_distribution_path = os.path.join(root_dir, 'target_class_distribution.pth').format(
    class_name)
target_class_distribution = torch.load(target_class_distribution_path)
bin_lows = target_class_distribution['histogram_bins']

match_bump_edits(
    data_loader=val_paths_data_loader,
    model=model,
    loss_fn=loss_fn,
    metric_fns=metric_fns,
    device=device,
    bin_lows=bin_lows,
    target_class_idx=target_class_idx,
    save_dir=os.path.join(os.path.dirname(config.save_dir), '{}_{}'.format(class_name, n_select)))

[0208_160948]***NEW ITR 1***


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:08<00:00, 31.26it/s]


assigned bin_idx 0
Cur bump_idx: 0 bump amount: 0.2 target_class_pred: 9248 bin_idx: 0
updating cur_bump_idx to 1, bump_amount to 0.25
bumps: [0.2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[0208_160957]***NEW ITR 2***


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:05<00:00, 45.80it/s]


bin_idx < cur_bump_idx (0 and 1) delta1=1.25 new bump_amount: 0.3125


...

100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:06<00:00, 44.76it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Overshot to bin 49. Decreasing bump amount to 6.457259657549906 aiming for bin 47
bumps: [0.2, 0.6103515625, 1.4901161193847656, 1.862645149230957, 2.3283064365386963, 2.6193447411060333, 2.9103830456733704, 3.092281986027956, 3.2741809263825417, 3.4788172342814505, 3.637978807091713, 3.751665644813329, 3.865352482534945, 4.092726157978177, 4.220623850414995, 4.348521542851813, 4.416467191958873, 4.547473508864641, 4.604316927725449, 4.7482018317168695, 4.89208673570829, 5.044964446199174, 5.115907697472721, 5.17985654369113, 5.341727060681478, 5.435651928564766, 5.503597577671826, 5.589591289822948, 5.675585001974071, 5.7199255098019925, 5.7553961596568115, 5.827338611652522, 5.847572426276315, 5.893256585856599, 5.9389407454368826, 5.962139732723745, 5.985338720010608, 6.03030906459745, 6.077420854164617, 6.115108419635362, 6.191547274880804, 6.213045702918585, 6.261585122472637, 6.310124542026688, 6.359422390011272, 6.385033127220829, 6.40720338113479, -1, 6.474820679613913, 6.50731

100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:08<00:00, 32.64it/s]

assigned bin_idx 47
Cur bump_idx: 47 bump amount: 6.457259657549906 target_class_pred: 51193 bin_idx: 47
updating cur_bump_idx to 48, bump_amount to 8.071574571937383
bumps: [0.2, 0.6103515625, 1.4901161193847656, 1.862645149230957, 2.3283064365386963, 2.6193447411060333, 2.9103830456733704, 3.092281986027956, 3.2741809263825417, 3.4788172342814505, 3.637978807091713, 3.751665644813329, 3.865352482534945, 4.092726157978177, 4.220623850414995, 4.348521542851813, 4.416467191958873, 4.547473508864641, 4.604316927725449, 4.7482018317168695, 4.89208673570829, 5.044964446199174, 5.115907697472721, 5.17985654369113, 5.341727060681478, 5.435651928564766, 5.503597577671826, 5.589591289822948, 5.675585001974071, 5.7199255098019925, 5.7553961596568115, 5.827338611652522, 5.847572426276315, 5.893256585856599, 5.9389407454368826, 5.962139732723745, 5.985338720010608, 6.03030906459745, 6.077420854164617, 6.115108419635362, 6.191547274880804, 6.213045702918585, 6.261585122472637, 6.310124542026688, 6


/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


In [ ]:
# Pedal to the metal!